In [2]:
import os
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
# import cv2
import pandas as pd
from PIL import Image, ImageFilter

import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

import tensorflow as tf

Using TensorFlow backend.


In [3]:
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend" #for GPU Support on MacBook
print(tf.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

1.13.1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16312467846782817705
]


In [4]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
currentDir = os.getcwd()
datasetDir = currentDir + "/FilteredImages/"
# datasetDir = currentDir + "/FilteredImages2/" # NEW SINGLE IMAGES DIRECTORY
trainDir = os.path.join(datasetDir, "train")
testDir = os.path.join(datasetDir, "test")
validDir = os.path.join(datasetDir, "valid")
y_dataDir = os.path.join(datasetDir, "y_data")

In [6]:
def sortFirst(val):
    return val[0]

def getBinary(dataFile):
    listOfTraining = []
    trainingHeader = []
    with open(dataFile) as csvfile:
#     reader = csv.DictReader(csvfile)
        reader2 = csv.reader(csvfile)
        listOfTraining = list(reader2)
        trainingHeader = listOfTraining[0]
        listOfTraining.pop(0)
#         listOfTraining = listOfTraining.sort(key = sortFirst, reverse = False)
#         np.asarray(listofTraining, dtype=np.int32)
#         return np.asarray(listofTraining, dtype=np.int32)
    listOfTrainingBinary = []
    for item in listOfTraining:
        listOfTrainingBinary.append(item[1])
    return np.asarray(listOfTrainingBinary, dtype=np.int32)

In [7]:
y_train = []
y_trainDir = os.path.join(y_dataDir, 'train_split_Depression_AVEC2017-edited.csv')
# print(y_trainDir)
y_train = getBinary(y_trainDir)
y_testDir = os.path.join(y_dataDir, 'dev_split_Depression_AVEC2017.csv')
# print(y_testDir)
y_test = getBinary(y_testDir)
print(y_train)
print(y_test)

# Y_train = np.asarray(y_train, dtype=np.int32)

[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0
 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0]


In [8]:
x_trainDir = trainDir
x_testDir = testDir

In [9]:
x_trainDir

'C:\\Users\\tanho\\Downloads\\Aaron-Workspace/FilteredImages/train'

In [10]:
# testSet = []
# testFolders =[]
# # trainingSet = []
# trainingFolders = []
# x_train = []
# y_train = []

def getImagesDir(mainDirectory):
    ImgDict = {}
    ImgList = []
    
    for folder in os.listdir(mainDirectory):
        theDir = os.path.join(mainDirectory, folder)
        images = os.listdir(theDir)
        listOfImgDir = []
        for img in images:
#             listOfImgDir.append(Image.open(os.path.join(theDir, img)))
            listOfImgDir.append(os.path.join(theDir, img))
#         print(listOfImgDir)
#         print(images)
        folderName = folder.split('_')
        ImgDict[folderName[0]] = listOfImgDir
        ImgList.append(listOfImgDir)
    imgList = ImgList.sort(key = sortFirst, reverse = False)
    return ImgList
        
        
#         for img in images
    
# for folder in toProcessList:
#     images = os.listdir(datasetDir + "/" + folder)
#     folderName = folder.split('_')
# #     print(folderName[0])
#     if folderName[0] in listOfTrainingName:
#         trainingFolders.append(datasetDir + "/" + folder)
# #         print(folderName)
#         index = listOfTrainingName.index(folderName[0])
#         temp = listOfTraining[index]
#         x_train.append(images)
#         tempBList = []
#         val = temp[1]
#         tempBList.append(val)
#         y_train.append(tempBList)
#         #temp.append(images)
#         #trainingSet.append(temp)
#     else:
#         testFolders.append(datasetDir + "/" + folder)
#         testSet.append(images)

trainingImagesDir = getImagesDir(trainDir)
testImagesDir = getImagesDir(testDir)
# np.array(trainingImages).shape

In [11]:
def importImages(listOfImgDir):
    myFolder = []
    for folder in listOfImgDir:
        myImages = []
        for image in folder:
            myImages.append(np.array(Image.open(image)))
        myFolder.append(np.array(myImages))
    return myFolder


#First Array iterate through Folder, Second Array Iterate though Image in Folder
trainingImages = importImages(trainingImagesDir)
testImages = importImages(testImagesDir)

In [12]:
#Num of Img, Height, Width
print(len(trainingImages)) 

len(y_train)

107


107

In [25]:
trainingImagesNP = np.array(trainingImages)
testImagesNP = np.array(testImages)

In [14]:
# print(trainingImages.shape)
# print(trainingImages[0])
# print(trainingImagesNP[10].shape)
# print(y_train)

neg = []
y_neg = []
pos = []
y_pos = []

for x,y in zip(trainingImages, y_train):
    if (y == 0):
        for each in x:
            neg.append(each)
            y_neg.append(0)
    else:
        for each in x:
            pos.append(each)
            y_pos.append(1)

X = pos + neg
Y = y_pos + y_neg

In [15]:
neg_test = []
y_neg_test = []
pos_test = []
y_pos_test = []

for x,y in zip(testImages, y_train):
    if (y == 0):
        for each in x:
            neg_test.append(each)
            y_neg_test.append(0)
    else:
        for each in x:
            pos_test.append(each)
            y_pos_test.append(1)

X_test = pos_test + neg_test
Y_test = y_pos_test + y_neg_test


npX_test = np.array(X_test)
npY_test = np.array(Y_test)
newNPX_test = npX_test.reshape(npX_test.shape[0], 101, 1000, 1)

In [16]:
vImagesDir = getImagesDir(validDir)
vImages = importImages(vImagesDir)
vImagesNP = np.array(vImages)

#TESTING DATA UNSEEN DATA
v = []
# y_neg_v = []
# pos_v = []
# y_pos_v = []

for x in vImages:
    for each in x:
        v.append(each)

# X_v = pos_v + neg_v
# Y_v = y_pos_v + y_neg_v


np_v = np.array(v)
# npY_test = np.array(Y_test)
newNPX_v = np_v.reshape(np_v.shape[0], 101, 1000, 1)

In [17]:
npX = np.array(X)
npY = np.array(Y)

In [18]:
print(npX.shape)
print(npY.shape)

(4591, 101, 1000)
(4591,)


In [19]:
newNPX = npX.reshape(npX.shape[0], 101, 1000, 1)

In [20]:
print(newNPX.shape)
print(newNPX_test.shape)
print(newNPX_v.shape)

(4591, 101, 1000, 1)
(1740, 101, 1000, 1)
(2381, 101, 1000, 1)


In [40]:
print(newNPX.dtype)
print(newNPX_test.dtype)
print(newNPX_v.dtype)

np.max(newNPX), np.max(newNPX_test)
newNPX = newNPX / np.max(newNPX)
newNPX_test = newNPX_test / np.max(newNPX_test)

np.max(newNPX), np.max(newNPX_test)

float64
float64
uint8


KeyboardInterrupt: 

In [ ]:
# print(trainingImagesNP)

In [ ]:
# input_shape = (101, 1000, 3)

# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=input_shape))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# # model.add(Dropout(0.5))
# model.add(Dense(1))
# model.add(Activation('sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.summary()




In [ ]:
# from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Saving Model
from keras.callbacks import ModelCheckpoint
numEpochs = 100
model_path = os.path.join(currentDir,'DAM-DHM-V-'+ str(numEpochs) +'.h5')

# checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True, mode='max')

# checkpoint = ModelCheckpoint(model_path, monitor='acc', verbose=1, save_best_only=True, mode='max')

checkpoint = ModelCheckpoint(model_path, monitor='acc', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, tbCallBack]

In [ ]:
# input_shape

In [ ]:
input_shape = (101, 1000, 1)

In [ ]:
# DR HARRY
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(AveragePooling2D(pool_size=(4, 3)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(AveragePooling2D(pool_size=(4, 3)))

model.add(Conv2D(96, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(96, (3, 3), padding='same', activation='relu'))
model.add(AveragePooling2D(pool_size=(4, 3)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adadelta', #adam
              metrics=['accuracy']) 

model.summary()

In [21]:
hist=model.fit(x=newNPX, y=npY, batch_size=32, epochs=numEpochs, callbacks=callbacks_list, validation_data=(newNPX_test, npY_test))

Train on 4591 samples, validate on 1740 samples
Epoch 1/100
4591/4591 [==============================] - 43s 9ms/step - loss: 0.6417 - acc: 0.7110 - val_loss: 0.7306 - val_acc: 0.5902

Epoch 00001: saving model to /home/ec2-user/workspace/Aaron-Workspace/DAM-DHM-V-100.h5
Epoch 2/100
4591/4591 [==============================] - 39s 9ms/step - loss: 0.6062 - acc: 0.7188 - val_loss: 0.7338 - val_acc: 0.5902

Epoch 00002: saving model to /home/ec2-user/workspace/Aaron-Workspace/DAM-DHM-V-100.h5
Epoch 3/100
4591/4591 [==============================] - 39s 9ms/step - loss: 0.5943 - acc: 0.7188 - val_loss: 0.6791 - val_acc: 0.5902

Epoch 00003: saving model to /home/ec2-user/workspace/Aaron-Workspace/DAM-DHM-V-100.h5
Epoch 4/100
4591/4591 [==============================] - 39s 9ms/step - loss: 0.5905 - acc: 0.7188 - val_loss: 0.7623 - val_acc: 0.5902

Epoch 00004: saving model to /home/ec2-user/workspace/Aaron-Workspace/DAM-DHM-V-100.h5
Epoch 5/100
4591/4591 [==============================] -

4591/4591 [==============================] - 39s 9ms/step - loss: 3.2495e-04 - acc: 0.9998 - val_loss: 4.6482 - val_acc: 0.6178

Epoch 00073: saving model to /home/ec2-user/workspace/Aaron-Workspace/DAM-DHM-V-100.h5
Epoch 74/100
4591/4591 [==============================] - 39s 8ms/step - loss: 3.0527e-04 - acc: 0.9998 - val_loss: 4.6881 - val_acc: 0.6207

Epoch 00074: saving model to /home/ec2-user/workspace/Aaron-Workspace/DAM-DHM-V-100.h5
Epoch 75/100
4591/4591 [==============================] - 39s 8ms/step - loss: 3.2246e-04 - acc: 0.9998 - val_loss: 4.6974 - val_acc: 0.6195

Epoch 00075: saving model to /home/ec2-user/workspace/Aaron-Workspace/DAM-DHM-V-100.h5
Epoch 76/100
4591/4591 [==============================] - 39s 9ms/step - loss: 3.2124e-04 - acc: 0.9998 - val_loss: 4.6936 - val_acc: 0.6178

Epoch 00076: saving model to /home/ec2-user/workspace/Aaron-Workspace/DAM-DHM-V-100.h5
Epoch 77/100
4591/4591 [==============================] - 39s 9ms/step - loss: 3.2111e-04 - acc: 

In [22]:
print("Model Accuracy: ", np.mean(hist.history['acc']))

Model Accuracy:  0.9511544325936078


In [23]:
model.save(''+ str(numEpochs) +' Epoch (Dr Harry Model) w validation.h5')

In [24]:
result = model.predict(newNPX_v)
for each in result:
    print(each)

[3.9469245e-14]
[0.00278088]
[1.]
[1.]
[2.2713786e-11]
[7.0725096e-14]
[0.0028819]
[1.]
[2.2140864e-07]
[2.4236947e-13]
[1.97043e-14]
[1.0658841e-15]
[1.]
[2.6435389e-11]
[1.0954466e-18]
[3.7639967e-07]
[8.616341e-06]
[2.3389987e-13]
[2.1257442e-11]
[2.7124929e-14]
[1.1794604e-17]
[2.2747776e-11]
[1.3839481e-10]
[0.9999982]
[1.6114015e-07]
[0.41187286]
[6.101588e-09]
[2.7292053e-09]
[1.1389953e-06]
[3.084887e-14]
[6.294132e-17]
[0.9756438]
[2.1389812e-16]
[1.1528816e-24]
[6.7408814e-17]
[1.7074464e-13]
[0.04829059]
[4.33071e-05]
[0.6420336]
[5.240428e-10]
[1.5743671e-08]
[0.00021128]
[1.6923971e-15]
[0.00174396]
[1.3634057e-12]
[0.00079264]
[1.03117884e-07]
[8.270933e-15]
[4.9035144e-08]
[2.1861526e-10]
[6.6145764e-09]
[5.2592863e-09]
[3.579414e-09]
[2.9613733e-19]
[4.6887014e-09]
[0.01652061]
[2.3124474e-15]
[1.1771751e-07]
[4.930294e-21]
[4.9680663e-12]
[8.1428175e-10]
[6.177817e-11]
[6.6842554e-09]
[1.0380757e-15]
[3.020135e-10]
[2.9536925e-06]
[5.7065896e-10]
[6.887361e-10]
[2.9811

[8.179482e-12]
[1.8219229e-21]
[2.1222568e-14]
[5.1554958e-17]
[8.429899e-33]
[4.6655943e-21]
[4.413164e-16]
[4.833932e-16]
[4.2201277e-22]
[1.3398865e-12]
[3.6564594e-19]
[5.700511e-13]
[2.7896529e-11]
[1.1373322e-22]
[1.7859847e-14]
[0.99503314]
[1.]
[6.0446835e-11]
[0.9998741]
[1.]
[0.99999905]
[4.6258756e-05]
[1.]
[1.6838534e-07]
[1.3068877e-08]
[0.05722134]
[4.5118286e-05]
[2.7891976e-13]
[0.00057228]
[1.]
[1.]
[0.01057316]
[2.0242157e-09]
[1.1072715e-10]
[7.956435e-10]
[1.0284987e-09]
[1.4041991e-07]
[0.9714929]
[0.00121597]
[3.1522447e-08]
[5.618844e-11]
[3.5615255e-08]
[6.634993e-05]
[2.2308894e-08]
[2.8509842e-12]
[1.]
[2.1866021e-11]
[9.867709e-07]
[0.30806118]
[7.666624e-08]
[0.002605]
[0.00031263]
[1.2080605e-08]
[1.]
[1.8448525e-09]
[0.9998647]
[3.2378722e-09]
[0.9635129]
[0.41755438]
[2.924549e-09]
[5.2737986e-10]
[2.7803614e-08]
[3.473801e-10]
[0.00043336]
[1.]
[5.6753997e-06]
[0.83689344]
[0.00011032]
[0.00065581]
[0.9440187]
[1.6616667e-07]
[0.9999994]
[1.8669347e-08]


[2.7843246e-16]
[1.]
[0.99997187]
[0.9999999]
[5.57053e-17]
[0.76141375]
[0.9821426]
[0.245375]
[1.]
[0.9999993]
[0.99999666]
[0.98886967]
[1.3196895e-09]
[0.99700314]
[8.54877e-09]
[0.99999964]
[1.]
[1.0272767e-05]
[1.]
[1.]
[1.]
[1.]
[0.00555142]
[1.]
[4.2635576e-11]
[0.9996942]
[1.0388766e-10]
[0.00044292]
[1.]
[1.2920138e-06]
[0.9999852]
[0.99999905]
[1.]
[1.]
[1.]
[0.9981323]
[1.]
[0.8211738]
[0.2923632]
[0.01808222]
[0.99999833]
[4.5088395e-09]
[0.99791807]
[3.8338768e-10]
[0.99840444]
[1.]
[0.00036026]
[3.6745732e-08]
[0.9999999]
[0.99967015]
[2.299117e-19]
[7.8888274e-10]
[8.5378997e-13]
[1.4547183e-09]
[1.04552e-10]
[4.957068e-06]
[7.847137e-14]
[0.00011736]
[1.720977e-12]
[1.3773294e-19]
[3.4945582e-12]
[9.806579e-13]
[1.2204244e-17]
[2.0275e-09]
[1.2774941e-10]
[1.09651905e-14]
[5.4461137e-19]
[4.1980436e-25]
[3.9830186e-19]
[2.973604e-12]
[6.4475153e-12]
[2.1167811e-12]
[4.4643464e-05]
[9.696212e-09]
[7.774043e-27]
[5.906751e-12]
[4.457716e-15]
[4.6421883e-13]
[4.7161962e-1